# 自作のweight matrixだと変数が吸収される場合がある

自作のweight matrixで単項式順序を決める場合以下の点に注意
- weight matrixの行ベクトルは線形独立でなくてはいけない
- weight matrixの各列のゼロではない最初の要素は，正でなくてなならない
- weight matrixの成分が実数である場合，sagemath(singular)によってintにキャストされている(と思われる)．よって，なるべく大きい値を行列の成分とする．

In [1]:
%pwd
%pip install --upgrade scipy
%pip install numpy==1.26.4
%pip install pulp

import numpy as np

load("/app/src/lib/sampler.sage")
load("/app/src/lib/buchberger.sage")
load("/app/src/lib/utils.sage")
load("/app/src/lib/differential_evolution.sage")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
params = np.array([3.1, 2.4, 1.7, 0.9, 0.5, -1.1, 0.3, -6.7, 3.2])
# フルランクかどうかの判定
variables = 3

float_matrix = params.reshape(variables, variables)
int_matrix = params.astype(int).reshape(variables, variables)

rank = np.linalg.matrix_rank(float_matrix, tol=0.1)
int_rank = np.linalg.matrix_rank(int_matrix)

print(float_matrix)
print(rank)

print(int_matrix)
print(int_rank)


[[ 3.1  2.4  1.7]
 [ 0.9  0.5 -1.1]
 [ 0.3 -6.7  3.2]]
3
[[ 3  2  1]
 [ 0  0 -1]
 [ 0 -6  3]]
3


In [140]:
M = np.random.uniform(0, 2, (3, 3))
M_list = M.tolist()

T = TermOrder(matrix(M_list))

print(M)
print('rank', np.linalg.matrix_rank(M, tol=0.1))
print()

M_int = M.astype(int)
print(M_int)
print('rank', np.linalg.matrix_rank(M_int))
print()
# # s = np.linalg.svdvals(M)
# print(s[0]/s[-1])

P = PolynomialRing(QQ, 3, 'x', order=T)
x0, x1, x2 = P.gens()

f = x0 + x1 + x2
print(f'now  {f}')
print(f'x0, x1, x2: {P(x0+1), P(x1+1),P(x2+1)}')
print('1 < x0:', 1 < x0, '1 < x1:', 1 < x1, '1 < x2:', 1 < x2)


print('validation:', validation(M_int))



[[0.96168041 1.75629485 1.03297767]
 [0.97880411 0.79039639 1.81482471]
 [1.3235664  0.40796727 0.1810715 ]]
rank 3

[[0 1 1]
 [0 0 1]
 [1 0 0]]
rank 3

now  x2 + x1 + x0
x0, x1, x2: (x0 + 1, x1 + 1, x2 + 1)
1 < x0: True 1 < x1: True 1 < x2: True
validation: True


In [4]:
P.term_order().matrix()

[   0.980265833370396   0.6652682528814213   0.3629002367809353]
[  0.6645093898032481 -0.13291052860851016  -0.2872649990344369]
[ -0.6160659131458779  0.25172741423172673  -0.3867440130836257]

In [5]:
vars(T)

{'_blocks': (),
 '_weights': (0.980265833370396, 0.6652682528814213, 0.3629002367809353),
 '_matrix': [   0.980265833370396   0.6652682528814213   0.3629002367809353]
 [  0.6645093898032481 -0.13291052860851016  -0.2872649990344369]
 [ -0.6160659131458779  0.25172741423172673  -0.3867440130836257],
 '_singular_moreblocks': 0,
 '_singular_ringorder_column': None,
 '_length': 3,
 '_name': 'matrix',
 '_singular_str': 'M(0,0,0,0,0,0,0,0,0)',
 '_macaulay2_str': '',
 '_magma_str': '"weight",[0,0,0,0,0,0,0,0,0]',
 '__doc__': '\nMatrix term order defined by a matrix A.\n\n`x^a < x^b` if and only if `x^{Aa} < x^{Ab}` where `<` is the lexicographic term order.\n'}

In [6]:
Mat = np.array([0,0,1,-1,0,0,0,0,3])
Mat = Mat.reshape(3,3)
np.linalg.matrix_rank(Mat, tol=0.001)

2

In [7]:
print(M)
np.sum(M[:,1:], axis=1)

[[ 0.98026583  0.66526825  0.36290024]
 [ 0.66450939 -0.13291053 -0.287265  ]
 [-0.61606591  0.25172741 -0.38674401]]


array([ 1.02816849, -0.42017553, -0.1350166 ])

In [8]:
print(vars(T))
print(T)
print(f)

{'_blocks': (), '_weights': (0.980265833370396, 0.6652682528814213, 0.3629002367809353), '_matrix': [   0.980265833370396   0.6652682528814213   0.3629002367809353]
[  0.6645093898032481 -0.13291052860851016  -0.2872649990344369]
[ -0.6160659131458779  0.25172741423172673  -0.3867440130836257], '_singular_moreblocks': 0, '_singular_ringorder_column': None, '_length': 3, '_name': 'matrix', '_singular_str': 'M(0,0,0,0,0,0,0,0,0)', '_macaulay2_str': '', '_magma_str': '"weight",[0,0,0,0,0,0,0,0,0]', '__doc__': '\nMatrix term order defined by a matrix A.\n\n`x^a < x^b` if and only if `x^{Aa} < x^{Ab}` where `<` is the lexicographic term order.\n'}
Matrix term order with matrix
[   0.980265833370396   0.6652682528814213   0.3629002367809353]
[  0.6645093898032481 -0.13291052860851016  -0.2872649990344369]
[ -0.6160659131458779  0.25172741423172673  -0.3867440130836257]
3*x0


In [9]:
vars(TermOrder('lex'))

{'_blocks': (),
 '_weights': None,
 '_matrix': None,
 '_singular_moreblocks': 0,
 '_singular_ringorder_column': None,
 '_length': 0,
 '_name': 'lex',
 '_singular_str': 'lp',
 '_macaulay2_str': 'Lex',
 '_magma_str': '"lex"',
 '__doc__': '\nLexicographic (lex) term order.\n\n`x^a < x^b` if and only if there exists `1 \\le i \\le n` such that `a_1 = b_1, \\dots, a_{i-1} = b_{i-1}, a_i < b_i`.\n'}

In [14]:
# P = PolynomialRing(QQ, variables, 'x', order=TermOrder(matrix_))
# x0, x1, x2 = P.gens()

m_lex = matrix([[1,0,0], [0,1,0], [0,0,1]])
T1 = TermOrder(m_lex)

matrix = randint(0, 1, (3, 3))
print(matrix)

# T = TermOrder(matrix([[1,1.5,1], [1,0.3,-1], [1,-1,1]]))

# P = PolynomialRing(QQ, 3, 'x', order=T)
# x0, x1, x2 = P.gens()
# polynomial_list = [-17/6*x0^2*x1*x2 + 17/13*x0*x2 + 17/14*x1^4*x2^5, 9/10*x0*x1*x2 + 3/10*x1^2 - 8/15*x1*x2^2 + 5/9*x1]
# print(f'now  {polynomial_list}')

TypeError: 'numpy.ndarray' object is not callable

In [11]:
# A = np.array([[1, 1, 4, 0, 1],
#               [0, 3, 1, 3, 2],
#               [1, 3, 0, 0, 1],
#               [2, 4, 3, 1, 1]]) # Aを定義する。

# np.linalg.matrix_rank(A) # Aのランクを計算する。
# type(A)

In [12]:
m_grevlex = matrix([[1,1,1], 
                    [0,0,-1], 
                    [0,-1,0]])


T3 = TermOrder(m_grevlex)
P_grevlex = PolynomialRing(QQ, 3, 'x', order=T3)
x0, x1, x2 = P_grevlex.gens()
polynomial_list = [-17/6*x0^2*x1*x2 + 17/13*x0*x2 + 17/14*x1^4*x2^5, 9/10*x0*x1*x2 + 3/10*x1^2 - 8/15*x1*x2^2 + 5/9*x1]
buchberger(P_grevlex, polynomial_list)

TypeError: 'numpy.ndarray' object is not callable